# Gaussian Results Aggregation

This pipeline combines the per-target **Gaussian estimator** results into session-level and then group-level datasets.

## Workflow

1. **Per-target results**
   - Each `(subject, session, target)` produces a `.pkl` result file.  
   - Expected: **23 results per session** (one per target node).  

2. **Session-level combination**
   - Check each `sub-*/ses-*` folder contains exactly 23 `.pkl` files.  
   - Combine them into a single `ResultsNetworkInference` object.  
   - Save as:  
     ```
     sub-XXX_ses-YYY_combined_gauss.pkl
     ```

3. **Group-level aggregation**
   - Metadata (`subject_session_metadata.csv`) provides group labels (Healthy, PD-off, PD-on).  
   - Collect all session-level results per group.  
   - Save one mega-`.pkl` per group:  
     ```
     Healthy_gauss_all_sessions.pkl
     PD-off_gauss_all_sessions.pkl
     PD-on_gauss_all_sessions.pkl
     ```

4. **Summary CSV**
   - Generate `gauss_results_summary.csv` listing subject, session, group, and file paths.  

## Outputs

- **Session combined files:** one `.pkl` per `(subject, session)`  
- **Group combined files:** one `.pkl` per group  
- **CSV summary:** `gauss_results_summary.csv`  

In [ ]:
import os
from pathlib import Path
import pickle
import pandas as pd
from idtxl.results import ResultsNetworkInference

# ----------------------------------------------------------------------
# 1. Path setup
# ----------------------------------------------------------------------
BASE_DIR = Path("/lustre/majlepy2/myproject")
RESULTS_DIR = BASE_DIR / "Results"
gauss_source_dir = BASE_DIR / "gaussian_mte_50"

# Directory where we save session-combined results
gauss_combined_dir = RESULTS_DIR / "gauss_results"
gauss_combined_dir.mkdir(exist_ok=True, parents=True)

# ----------------------------------------------------------------------
# 2. Load metadata (subject/session info)
# ----------------------------------------------------------------------
meta = pd.read_csv(BASE_DIR / "subject_session_metadata.csv")
meta['sub_ses'] = meta['subject'] + '/' + meta['session']

# ----------------------------------------------------------------------
# 3. Scan session folders for 23 target .pkl files
# ----------------------------------------------------------------------
session_dirs = []
for subses_dir in sorted(gauss_source_dir.glob("sub-*/ses-*")):
    pkl_files = list(subses_dir.glob("*.pkl"))
    if len(pkl_files) == 23:
        session_dirs.append(subses_dir)
    else:
        print(f"Skipping {subses_dir}: found {len(pkl_files)} pkl files (expected 23)")
print(f"Found {len(session_dirs)} sessions with 23 targets.")

# ----------------------------------------------------------------------
# 4. Check a combined file (optional sanity check)
# ----------------------------------------------------------------------
with open(combined_path, "rb") as f:
    comb = pickle.load(f)
print("Combined object type:", type(comb))
print("Targets analyzed:", comb.targets_analysed)

# ----------------------------------------------------------------------
# 5. Organize results by group and export
# ----------------------------------------------------------------------
RESULTS_DIR = Path("/lustre/majlepy2/myproject/Results")
gauss_dir = RESULTS_DIR / "gauss_results"

meta = pd.read_csv("/lustre/majlepy2/myproject/subject_session_metadata.csv")
meta['sub_ses'] = meta['subject'] + '_' + meta['session']
group_dict = dict(zip(meta['sub_ses'], meta['group']))

combined_files = sorted(gauss_dir.glob("*_combined_gauss.pkl"))

group_results = {}     # { group_name: [list of session Results objects] }
summary_rows = []      # metadata summary for CSV export

for f in combined_files:
    fname = f.name
    try:
        stem = fname.replace("_combined_gauss.pkl", "")
        subj, sess = stem.split("_", 1)
        sub_ses = f"{subj}_{sess}"
    except Exception as e:
        print(f"Skipping {fname}: couldn't parse subject/session ({e})")
        continue

    group = group_dict.get(sub_ses)
    if group is None:
        print(f"WARNING: {sub_ses} not found in metadata.")
        continue

    with open(f, "rb") as pf:
        res = pickle.load(pf)

    group_results.setdefault(group, []).append(res)
    summary_rows.append({
        "file": str(f),
        "subject": subj,
        "session": sess,
        "group": group
    })

# ----------------------------------------------------------------------
# 6. Save aggregated group files and summary CSV
# ----------------------------------------------------------------------
for group, results_list in group_results.items():
    out_path = gauss_dir / f"{group}_gauss_all_sessions.pkl"
    with open(out_path, "wb") as pf:
        pickle.dump(results_list, pf)
    print(f"Saved {group}: {out_path} ({len(results_list)} sessions)")

summary_df = pd.DataFrame(summary_rows)
summary_csv = gauss_dir / "gauss_results_summary.csv"
summary_df.to_csv(summary_csv, index=False)
print(f"Saved session-to-group mapping: {summary_csv}")
